**Show all your work for full credit. Each source code you submit should include detailed comments and instructions on how to run it in order to confirm that it works as expected. If the program that does not run or throws runtime errors, it cannot be graded. You can refer to the programming guidelines from the TAs here: https://tinyurl.com/CPEG-472-672-Programming-Guide/**

**This is an individual assignment and each student should work on their own. Ensure you don't share any code online or with others (note, using Replit, GitHub and similar online platforms can make your code accessible to others).**

**To submit the assignment, you need to use Jupyter Notebook with the provided cell blocks and follow the naming conventions and instructions posted here: https://tinyurl.com/CPEG-472-672-Programming-Guide/**

Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel $\rightarrow$ Restart) and then **run all cells** (in the menubar, select Cell $\rightarrow$ Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and section below:

In [1]:
NAME = "Shruthilaya Arun"
#SECTION = "472"
SECTION = "672"

---

## <font color='blue'> Question 4 [25 points total - answer all parts] 

In [2]:
 !pip install simonspeckciphers

In [3]:
# !pip install simonspeckciphers
from simon import SimonCipher

## <font color='blue'>(a) [10 points] Implement a M-D hash H based on the D-M compression that uses the Simon128/128 block cipher (example library implementation https://pypi.org/project/simonspeckciphers/). The M-D hash should be able to access blocks of arbitrary length and use proper padding.</font>

In [5]:

# 3 points
def padding(ptxt: bytes) -> bytes:
    """
    You must padd the ptxt to a multiple of 16, based on 
    the padding blueprint asked in the question. return the padded ptxt in
    bytes format.
    """
    
    padded_ptxt = b""
    # YOUR CODE HERE
    if len(ptxt) % 16 == 0:
        return ptxt
    padded_ptxt = ptxt + b'\x80' # add 1
    pad_len = (16 - (len(padded_ptxt) % 16)) -1 #get length
    padded_ptxt += b'\x00' * pad_len # add zeros
    original_length = len(ptxt).to_bytes(1, byteorder='big')  # get actual length
    padded_ptxt += original_length

    return padded_ptxt
# 3 points
def compression(ptxt: int, key: int) -> int:
    """ 
    Implement the D-M compression using the Simon encryption. The output hash h 
    should be an integer.
    """
    h = 0
    # YOUR CODE HERE
    cipher = SimonCipher(key) # initialise key
    encryption= cipher.encrypt(ptxt) # encrypt ptxt
    h=ptxt^encryption # xor ptxt and encrypted ptxt
    return h
# 4 points
def merkel_damgard(ptxt: bytes) -> str:
    """
    input is ptxt in bytes format. Implement the merkel_damgard hash that uses padding
    function to pad the ptxt, and uses compression function as it's D-M compression.
    You should return the digest in hex string format.
    """
    digest = "" #hex string
    # YOUR CODE HERE
    padded_ptxt = padding(ptxt)
    h=0
    for i in range(0, len(padded_ptxt), 16):
        block = int.from_bytes(padded_ptxt[i:i + 16], byteorder='big')  # get block of ptxt
        h = compression(h , block)  # compression
    digest=hex(h) # convert to hex 
    return digest[2:] # remove 0x prefix

In [6]:
ptxt1 = b"fourfourfourfour"
padded_ptxt1 = padding(ptxt1)
assert type(padded_ptxt1) == bytes
assert padded_ptxt1 == b'fourfourfourfour'

ptxt2 = b"thisis a long random string that you need to pad try"
padded_ptxt2 = padding(ptxt2)
assert type(padded_ptxt2) == bytes
assert padded_ptxt2 == b'thisis a long random string that you need to pad try\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x004'

In [7]:
ptxt3 = b"thisismyplace"
padded_ptxt3 = padding(ptxt3)
assert type(padded_ptxt3) == bytes
assert padded_ptxt3 == b'thisismyplace\x80\x00\r'

In [8]:
ptxt1 = 254525115973874115874502719046257766467
key1 = 75587892947578446201832214508467310358
h1 = compression(ptxt1,key1)
assert type(h1) == int
assert h1 == 319822978455615316176439425277408331349

ptxt2 = 101762340182218496348316498306555754546
key2 = 31343182667721937931820629963621645992
h2 = compression(ptxt2,key2)
assert type(h2) == int
assert h2 == 11086398588071857704075469178241880169

In [9]:
ptxt3 = 0x0000000000000000
key3 = 154732585181220405374602665520340664333
h2 = compression(ptxt3, key3)
assert h2 == 306284589063070061155210186206098569823


In [10]:
digest1 = merkel_damgard(b"thisismyplace")
assert type(digest1) == str
assert len(digest1) == 32
assert digest1 == "e66c4422efc6a7678bcfde4e6add3a5f"

digest1 = merkel_damgard(b"justaranodmstringlielijfle")
assert type(digest1) == str
assert len(digest1) == 32
assert digest1 == "184e29e1c7affead1f3f9f6be97acc9d"

## <font color='blue'> (b) [15 points] Implement a secret-prefix MAC based on H from the previous step as illustrated in the reading and the lecture slides. Then, given a valid MAC T1 of a message M1 under a secret key K, implement a program that generates a valid forgery T2 for message M2 under the same key; in this case, M1 is the leftmost substring of M2 (i.e., M2 begins with the same bytes as M1, and M2 is longer than M1). Assume the MAC key K, and messages M1 and M2 have a bitsize that is an exact multiple of the block size of H. Demonstrate correctness by showing you can generate a valid pair (T2,M2) without knowing K, given M1, M2 and (T1,M1).</font>

In [11]:
# 5 points
def prefix_MAC(key: bytes, M1: bytes) -> str:
    """ 
    Implement a secret-prefix MAC using hash function implemented in part a.
    inputs are in bytes. Return tag T1 of message key+M1 in hex string format.
    """ 
    T1 = ""
    # YOUR CODE HERE
    combined = key + M1 # message
    T1 = merkel_damgard(combined)  #tag
    
    return T1
    
# 10 points
def forgery(T1: str, M2:bytes) -> str:
    """
    Given a valid MAC tag T1 of a message M1, and a message M2, can you forge
    a new valid tag for message M1+M2? T1 is in hex string format and M2 is bytes.
    Return the new tag in hex string format.
    """
    T2 = ""
    # YOUR CODE HERE
    T1_int = int(T1,16) # convert to int
    M2_int=int.from_bytes(M2,'big') # from bytes to int
    T2_int=compression(T1_int,M2_int) #compression 
    T2=hex(T2_int)
    return T2[2:]

In [12]:
key = b"\xc6\x80\x08\xdb\x8c\x06'\nB\x84\x15,\xf0\xd7i:"
M1 = b'\xe8\xf5\x99\xaa\xaf\xe4b\x83y\x9a,l\x9b\xfb\xe6\xeb'
T1 = prefix_MAC(key, M1)
assert T1 == "73b2bd3d98ccca2c0ba8e5c4589709b2"

In [13]:
T1 = "5961a52451d35761f6a68ca9142c9a43"
M2 = b'!)\xe4\xcd\xd1@@\xb2d\xd6\x18\xc7?\xa6=m'
T2 = forgery(T1,M2)
assert T2 == "15b89ee0658205e22146306df98a1eab"

In [14]:
# hidden tests